Use adaboost, Gradient Tree Boosting and xgboosting

# NOTES:
Der er nogle ting du skal tilbage og havde styr på. 

Baglens:

- Du skal have flere variabler med herover den først runde feature selection skal kun være helt grov ect vægle mellem log eller ike log ect.
- Der er pludseligt en helt masse missing i dit y!? wth! får lige styr på den kat..
- i forgående gem df som goepandas.dataframe i pickle
- i forgående gem df (- geometry) som padas.dataframe i csv
- ændre navn fatilities i forgående og lav også en logged version.
- Se generelt din estimation for hvad der giver mening som logged...

forlens:

- 1000 random less-unbalanced xgboosts to get a distribution of metric results
- tree graph
- world map of predict vs true. (proba vs log(best) t+1 or pred vs binary_best t +1)
- world map of diff between binary pred and true.
- Use the true hold-out-set.
- Robustness test with only new onsets
- you can easely do basian correction: even country (or something) specific

$$P(død)=\frac{xgboost\_est\_(dist) * country\_mean }{xgboost\_est\_(dist) * country\_mean + (1-xgboost\_est\_(dist)) * (1-country\_mean) }$$

men hvis du vil gøre det med dists kan du vel ikke bruge $1-$..

- det må være et empirisk spørgsmål om 1/1, 1/30, 1/1 med B-correction eller 1/30 med B-correction er bedst.

- tjek if your model gets worse the more years we move away from 2005

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import pickle

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb 
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, average_precision_score, recall_score, precision_recall_curve, roc_auc_score, roc_curve, precision_score, accuracy_score
from sklearn.utils.fixes import signature


In [2]:

pkl_file = open('full_df.pkl', 'rb')

df = pickle.load(pkl_file)

pkl_file.close()

df.dropna(inplace = True) # do to some unknown fuckery in the last script.. to fix later

In [3]:
df.shape

(1267740, 62)

In [4]:
X_labels = ['past_fatalities', 'fatilities_country_year', 'nearest_conflict','past_conflicts',
            'landarea','country_area_sum',
            'interp_pop_gpw_sum','country_pop_sum',         
            'light_capita_cell','light_capita_country',
            'low_ratio_light', 'diff_median_light',
            'mountains_mean', 'ttime_mean', 'capdist',
            'bdist1',
            'excluded_binary',
            'petroleum_full']

# The functions:

**Du mangler stadigt at gemme din "posterior" aka alle forudsiagte y_tilde (i out_years modellerne.)**

In [5]:
# dict of metrics

def cons_dict_of_metric():
    # Construct the empty dict of matrics and other stuff
    dict_of_metrics = {}
    dict_of_metrics['auc'] = []
    dict_of_metrics['ap'] = []

    dict_of_metrics['recall50'] = []
    dict_of_metrics['recall75'] = []
    dict_of_metrics['recall95'] = []

    dict_of_metrics['precision50'] = []
    dict_of_metrics['precision75'] = []
    dict_of_metrics['precision95'] = []
    
    dict_of_metrics['accuracy50'] = []
    
    #roc,rp, feature imp
    
    dict_of_metrics['roc'] = []
    dict_of_metrics['prc'] = []
    dict_of_metrics['feature_imp'] = []
    
    return(dict_of_metrics)

In [6]:
def out_years_estimation(sample_size = 100, balance_parameter = 1):    
    
    # Hold out years:
    df_test_years = df[df['year']>2005].copy()
    df_train_years = df[df['year']<=2005].copy()

    #radnom seeds for loop (culd do without, but it eases intpr.)
    random_seeds = np.random.randint(1000,9999,1000)

    # dict of metrics
    dict_of_metrics = cons_dict_of_metric()    
    dict_of_metrics['preds1'] = []
    dict_of_metrics['preds0'] = []

    y_label = 'lead_binary_best'

    df_train_years_1 = df_train_years[df_train_years['lead_binary_best'] == 1] # events


    #for loop
    for i in range(sample_size):

        # The data:
        # the non_events! rigth now 1000 percent
        df_train_years_0 = df_train_years[df_train_years['lead_binary_best'] == 0].sample(df_train_years_1.shape[0]*balance_parameter, random_state = random_seeds[i]) 
        # you shoul dmeak one with *1 and *20 manke it an input in the function 

        #Merge events and non-events; rigth now 10/100
        df_train_years_01 = pd.concat([df_train_years_0,df_train_years_1])

        # dfine X and y
        X_train = df_train_years_01[X_labels]

        y_train = df_train_years_01[y_label]

        # test_train: TEMP:
        # Create the training and test sets
        #X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=random_seeds[i])

        X_test = df_test_years[X_labels]
        y_test = df_test_years[y_label]

        # the model
        # Instantiate the XGBClassifier: xg_cl
        xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, seed=random_seeds[i])

        # Fit the classifier to the training set
        xg_cl.fit(X_train, y_train)

        # Predict the labels of the test set: preds
        preds = xg_cl.predict_proba(X_test)

        preds50 = (preds[:,1] >= 0.5)*1 # binary class with thresshold at 0.5 (standart)
        preds75 = (preds[:,1] >= 0.75)*1 # binary class with thresshold at 0.75 (standart)
        preds95 = (preds[:,1] >= 0.95)*1 # binary class with thresshold at 0.95 (standart)

        dict_of_metrics['ap'].append(average_precision_score(y_test,preds[:,1]))
        dict_of_metrics['auc'].append(roc_auc_score(y_test,preds[:,1]))

        dict_of_metrics['recall50'].append(recall_score(y_test, preds50))
        dict_of_metrics['recall75'].append(recall_score(y_test, preds75))
        dict_of_metrics['recall95'].append(recall_score(y_test, preds95))

        dict_of_metrics['precision50'].append(precision_score(y_test, preds50))
        dict_of_metrics['precision75'].append(precision_score(y_test, preds75))
        dict_of_metrics['precision95'].append(precision_score(y_test, preds95))
        
        dict_of_metrics['accuracy50'].append(accuracy_score(y_test,preds50))
        
        dict_of_metrics['prc'].append(precision_recall_curve(y_test,preds[:,1]))
        dict_of_metrics['roc'].append(roc_curve(y_test, preds[:, 1]))
    
        dict_of_metrics['feature_imp'].append(xg_cl.feature_importances_)
        
        dict_of_metrics['preds1'].append(preds[:,1]) # nice to have both columns for the baysian correction
        dict_of_metrics['preds0'].append(preds[:,0]) # nice to have both columns for the baysian correction

        print(i, end='_')

    return(dict_of_metrics)

In [7]:
def out_years_estimation_onsets(sample_size = 100, balance_parameter = 1):   
    
    df_ongoing = df[(df['lead_binary_best']==1)&(df['binary_best'] == 1)]
    df_onset = df[~((df['lead_binary_best']==1)&(df['binary_best'] == 1))]
    
    print('Number of observation/conflicts discarded: {}'.format(df_ongoing.shape[0]))
    print('Number of observation preserved: {}'.format(df_onset.shape[0]))
    print('Number of conflicts preserved: {}'.format(df_onset[df_onset['lead_binary_best'] == 1].shape[0]))
    
    # Hold out years:
    df_test_years = df_onset[df_onset['year']>2005].copy()
    df_train_years = df_onset[df_onset['year']<=2005].copy()

    #radnom seeds for loop (culd do without, but it eases intpr.)
    random_seeds = np.random.randint(1000,9999,1000)

    # dict of metrics
    dict_of_metrics = cons_dict_of_metric()    
    dict_of_metrics['preds1'] = []
    dict_of_metrics['preds0'] = []


    y_label = 'lead_binary_best'
    
    
    df_train_years_1 = df_train_years[df_train_years['lead_binary_best'] == 1] # events


    #for loop
    for i in range(sample_size):

        # The data:
        # the non_events! rigth now 1000 percent
        df_train_years_0 = df_train_years[df_train_years['lead_binary_best'] == 0].sample(df_train_years_1.shape[0]*balance_parameter, random_state = random_seeds[i]) 
        # you shoul dmeak one with *1 and *20 manke it an input in the function 

        #Merge events and non-events; rigth now 10/100
        df_train_years_01 = pd.concat([df_train_years_0,df_train_years_1])

        # dfine X and y
        X_train = df_train_years_01[X_labels]

        y_train = df_train_years_01[y_label]

        # test_train: TEMP:
        # Create the training and test sets
        #X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=random_seeds[i])

        X_test = df_test_years[X_labels]
        y_test = df_test_years[y_label]

        # the model
        # Instantiate the XGBClassifier: xg_cl
        xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, seed=random_seeds[i])

        # Fit the classifier to the training set
        xg_cl.fit(X_train, y_train)

        # Predict the labels of the test set: preds
        preds = xg_cl.predict_proba(X_test)

        preds50 = (preds[:,1] >= 0.5)*1 # binary class with thresshold at 0.5 (standart)
        preds75 = (preds[:,1] >= 0.75)*1 # binary class with thresshold at 0.75 (standart)
        preds95 = (preds[:,1] >= 0.95)*1 # binary class with thresshold at 0.95 (standart)

        dict_of_metrics['ap'].append(average_precision_score(y_test,preds[:,1]))
        dict_of_metrics['auc'].append(roc_auc_score(y_test,preds[:,1]))

        dict_of_metrics['recall50'].append(recall_score(y_test, preds50))
        dict_of_metrics['recall75'].append(recall_score(y_test, preds75))
        dict_of_metrics['recall95'].append(recall_score(y_test, preds95))

        dict_of_metrics['precision50'].append(precision_score(y_test, preds50))
        dict_of_metrics['precision75'].append(precision_score(y_test, preds75))
        dict_of_metrics['precision95'].append(precision_score(y_test, preds95))

        dict_of_metrics['accuracy50'].append(accuracy_score(y_test,preds50))
        
        dict_of_metrics['prc'].append(precision_recall_curve(y_test,preds[:,1]))
        dict_of_metrics['roc'].append(roc_curve(y_test, preds[:, 1]))
    
        dict_of_metrics['feature_imp'].append(xg_cl.feature_importances_)
        
        #dict_of_metrics['preds'].append(preds) # nice to have both columns for the baysian correction
        dict_of_metrics['preds1'].append(preds[:,1]) # nice to have both columns for the baysian correction
        dict_of_metrics['preds0'].append(preds[:,0]) # nice to have both columns for the baysian correction
        
        print(i, end='_')

    return(dict_of_metrics)

# Estimates:

But you still need to go back and optimize the hyper parameters. But that is not paramount for now.

In [8]:
dict_of_metrics_out_years = out_years_estimation(1000)

file_name = "dict_of_metrics_out_years.pkl"
output = open(file_name, 'wb') #
pickle.dump(dict_of_metrics_out_years, output)
output.close()

del(dict_of_metrics_out_years)

0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28_29_30_31_32_33_34_35_36_37_38_39_40_41_42_43_44_45_46_47_48_49_50_51_52_53_54_55_56_57_58_59_60_61_62_63_64_65_66_67_68_69_70_71_72_73_74_75_76_77_78_79_80_81_82_83_84_85_86_87_88_89_90_91_92_93_94_95_96_97_98_99_100_101_102_103_104_105_106_107_108_109_110_111_112_113_114_115_116_117_118_119_120_121_122_123_124_125_126_127_128_129_130_131_132_133_134_135_136_137_138_139_140_141_142_143_144_145_146_147_148_149_150_151_152_153_154_155_156_157_158_159_160_161_162_163_164_165_166_167_168_169_170_171_172_173_174_175_176_177_178_179_180_181_182_183_184_185_186_187_188_189_190_191_192_193_194_195_196_197_198_199_200_201_202_203_204_205_206_207_208_209_210_211_212_213_214_215_216_217_218_219_220_221_222_223_224_225_226_227_228_229_230_231_232_233_234_235_236_237_238_239_240_241_242_243_244_245_246_247_248_249_250_251_252_253_254_255_256_257_258_259_260_261_262_263_264_265_266_267_268_269_270_271_272_273_274_275_276_27

In [9]:
dict_of_metrics_out_years_onsets = out_years_estimation_onsets(1000)

file_name = "dict_of_metrics_out_years_onsets.pkl"
output = open(file_name, 'wb') #
pickle.dump(dict_of_metrics_out_years_onsets, output)
output.close()

del(dict_of_metrics_out_years_onsets)

Number of observation/conflicts discarded: 6887
Number of observation preserved: 1260853
Number of conflicts preserved: 6410
0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28_29_30_31_32_33_34_35_36_37_38_39_40_41_42_43_44_45_46_47_48_49_50_51_52_53_54_55_56_57_58_59_60_61_62_63_64_65_66_67_68_69_70_71_72_73_74_75_76_77_78_79_80_81_82_83_84_85_86_87_88_89_90_91_92_93_94_95_96_97_98_99_100_101_102_103_104_105_106_107_108_109_110_111_112_113_114_115_116_117_118_119_120_121_122_123_124_125_126_127_128_129_130_131_132_133_134_135_136_137_138_139_140_141_142_143_144_145_146_147_148_149_150_151_152_153_154_155_156_157_158_159_160_161_162_163_164_165_166_167_168_169_170_171_172_173_174_175_176_177_178_179_180_181_182_183_184_185_186_187_188_189_190_191_192_193_194_195_196_197_198_199_200_201_202_203_204_205_206_207_208_209_210_211_212_213_214_215_216_217_218_219_220_221_222_223_224_225_226_227_228_229_230_231_232_233_234_235_236_237_238_239_240_241_242_243_244_245_2

In [10]:
dict_of_metrics_out_years20 = out_years_estimation(1000, 20)

file_name = "dict_of_metrics_out_years20.pkl"
output = open(file_name, 'wb') #
pickle.dump(dict_of_metrics_out_years20, output)
output.close()

del(dict_of_metrics_out_years20)

0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28_29_30_31_32_33_34_35_36_37_38_39_40_41_42_43_44_45_46_47_48_49_50_51_52_53_54_55_56_57_58_59_60_61_62_63_64_65_66_67_68_69_70_71_72_73_74_75_76_77_78_79_80_81_82_83_84_85_86_87_88_89_90_91_92_93_94_95_96_97_98_99_100_101_102_103_104_105_106_107_108_109_110_111_112_113_114_115_116_117_118_119_120_121_122_123_124_125_126_127_128_129_130_131_132_133_134_135_136_137_138_139_140_141_142_143_144_145_146_147_148_149_150_151_152_153_154_155_156_157_158_159_160_161_162_163_164_165_166_167_168_169_170_171_172_173_174_175_176_177_178_179_180_181_182_183_184_185_186_187_188_189_190_191_192_193_194_195_196_197_198_199_200_201_202_203_204_205_206_207_208_209_210_211_212_213_214_215_216_217_218_219_220_221_222_223_224_225_226_227_228_229_230_231_232_233_234_235_236_237_238_239_240_241_242_243_244_245_246_247_248_249_250_251_252_253_254_255_256_257_258_259_260_261_262_263_264_265_266_267_268_269_270_271_272_273_274_275_276_27

In [11]:
dict_of_metrics_out_years_onsets5 = out_years_estimation_onsets(1000,5)

file_name = "dict_of_metrics_out_years_onsets5.pkl"
output = open(file_name, 'wb') #
pickle.dump(dict_of_metrics_out_years_onsets5, output)
output.close()

del(dict_of_metrics_out_years_onsets5)

Number of observation/conflicts discarded: 6887
Number of observation preserved: 1260853
Number of conflicts preserved: 6410
0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28_29_30_31_32_33_34_35_36_37_38_39_40_41_42_43_44_45_46_47_48_49_50_51_52_53_54_55_56_57_58_59_60_61_62_63_64_65_66_67_68_69_70_71_72_73_74_75_76_77_78_79_80_81_82_83_84_85_86_87_88_89_90_91_92_93_94_95_96_97_98_99_100_101_102_103_104_105_106_107_108_109_110_111_112_113_114_115_116_117_118_119_120_121_122_123_124_125_126_127_128_129_130_131_132_133_134_135_136_137_138_139_140_141_142_143_144_145_146_147_148_149_150_151_152_153_154_155_156_157_158_159_160_161_162_163_164_165_166_167_168_169_170_171_172_173_174_175_176_177_178_179_180_181_182_183_184_185_186_187_188_189_190_191_192_193_194_195_196_197_198_199_200_201_202_203_204_205_206_207_208_209_210_211_212_213_214_215_216_217_218_219_220_221_222_223_224_225_226_227_228_229_230_231_232_233_234_235_236_237_238_239_240_241_242_243_244_245_2